In [1]:
import pandas as pd
#import ast
import json

In [95]:
file = open('StreamToS3-5-2017-10-31-00-44-50-f5137ee8-1e99-425e-a530-ea95e18a98cd')

In [96]:
# JSON-ify the input
text = file.read()

text = '[' + text + ']'

text = text.replace('}{', '},{')

In [99]:
df = pd.read_json(text)

In [47]:
#split_text = text.replace('}{', '}:::::{').split(':::::')

In [48]:
#split_text_json = [json.loads(_) for _ in split_text]

In [49]:
#df = pd.DataFrame.from_dict(split_text_json, orient='columns')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 10 columns):
authorKey        13130 non-null float64
contentKey       13130 non-null float64
pathRoot         13194 non-null object
publishedDate    13130 non-null object
referrer         11292 non-null object
siteKey          13194 non-null int64
timeStamp        13194 non-null datetime64[ns]
userAgent        13194 non-null object
userId           13036 non-null object
userStatus       13194 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 1.0+ MB


In [11]:
df.publishedDate = pd.to_datetime(df.publishedDate)

In [15]:
df.referrer.unique()

array(['https://www.facebook.com/',
       'https://247sports.com/nfl/pittsburgh-steelers/Gallery/Survey-lists-most-popular-Halloween-candy-in-every-US-State-109726346',
       'http://m.facebook.com/', None, 'http://m.facebook.com',
       'https://m.facebook.com/home.php',
       'https://247sports.com/nfl/carolina-panthers/Gallery/Ranking-the-NFLs-defenses-through-Week-8-109717680',
       'https://247sports.com/nfl/green-bay-packers/Gallery/Survey-lists-most-popular-Halloween-candy-in-every-US-State-109726346',
       'https://m.facebook.com/', 'android-app://com.google.android.gm',
       'android-app://m.facebook.com', 'https://www.google.com/',
       'https://247sports.com/nfl/carolina-panthers/Gallery/Survey-lists-most-popular-Halloween-candy-in-every-US-State-109726346/9',
       'https://www.cbssports.com/nfl/teams/page/PIT/pittsburgh-steelers',
       'https://247sports.com/nfl/new-england-patriots/Gallery/The-actor-best-fit-to-play-each-NFL-quarterback-for-Halloween--45932

In [16]:
import re

In [33]:
def parse_team(row):
    if row.referrer is None: return None
    team = re.search(r'247sports.com\/nfl\/([^/]+)\/', row.referrer)
    if team is None: return None
    return team.group(1)

In [100]:
df['team'] = df.apply(parse_team, axis = 1)

In [109]:
set(df.siteKey.unique()).difference(set(WRONG.values()))

{25296896}

In [ ]:
df.loc[~df.team.isna(), ['team', 'siteKey']].groupby(['team', 'siteKey']).team.count()

In [17]:
def parse_domain(row):
    if row.referrer is None: return None
    if row.referrer.count('/') < 3: return re.search(r'\/\/([^/]+)', row.referrer).group(1)
    return re.search(r'\/\/([^/]+)\/', row.referrer).group(1)

In [18]:
df.loc[22, 'referrer']

'android-app://com.google.android.gm'

In [19]:
df['referrer_domain'] = df.apply(parse_domain, axis = 1)

In [20]:
df.shape

(12918, 11)

In [21]:
df.loc[df.referrer_domain.isna()].shape

(1916, 11)

In [22]:
df.groupby('referrer_domain').referrer_domain.count()

referrer_domain
247sports.com                               563
com.google.android.gm                        93
com.google.android.googlequicksearchbox      10
fanly.link                                    7
l.facebook.com                               11
lm.facebook.com                              29
m.facebook.com                             9519
news.google.com                               2
news.url.google.com                           2
outlook.live.com                             13
r.search.yahoo.com                            3
scout.com                                     1
search.myway.com                              1
t.co                                          9
www.bing.com                                  2
www.cbssports.com                            31
www.facebook.com                            446
www.fark.com                                  2
www.google.ca                                 7
www.google.com                              228
www.google.com.au       

In [23]:
df.loc[df.userAgent.str.contains('WOW64')]

,authorKey,contentKey,pathRoot,publishedDate,referrer,siteKey,timeStamp,userAgent,userId,userStatus,referrer_domain
55,1.000669e+09,3.735750e+09,BOLT,2017-10-26 22:15:51,https://www.facebook.com/,24051712,2017-10-31 00:59:53,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,cf319687-f1a3-4c2c-bc04-ad3ae9488bd4,Guest,www.facebook.com
240,2.984837e+09,1.041827e+09,BOLT,2017-10-29 13:56:02,https://www.facebook.com/,24248320,2017-10-31 00:59:58,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,bfb0ce93-fecd-4660-a51f-6d3bd4850e24,Guest,www.facebook.com
428,4.179755e+09,3.242395e+09,BOLT,2017-10-30 08:02:13,https://www.facebook.com/,24444928,2017-10-31 01:00:02,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,bae63d97-1bc7-4fcc-9b40-41b09ffa1fb2,Guest,www.facebook.com
817,2.984837e+09,1.041827e+09,BOLT,2017-10-29 13:56:02,https://www.facebook.com/,24444928,2017-10-31 01:00:13,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,7958c893-da5d-4f01-b52e-c9ede2b20816,Guest,www.facebook.com
930,2.605384e+09,1.281296e+09,BOLT,2017-10-30 19:18:19,None,24248320,2017-10-31 01:00:13,Mozilla/5.0 (Windows NT 10.0; WOW64; rv:56.0) ...,66ed2ef2-9b45-4acf-a955-ec9eab2c3668,Guest,None
1337,2.605384e+09,1.281296e+09,BOLT,2017-10-30 19:18:19,None,24248320,2017-10-31 01:00:15,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,abaf1245-d12e-41a7-be9b-f45ef5f400a8,Guest,None
1511,1.299448e+09,1.251739e+09,BOLT,2017-10-30 10:31:13,None,24379392,2017-10-31 01:00:23,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:56.0) G...,d5739cfb-2d65-46dd-86ba-bb1d658ceee6,Guest,None
1645,2.216100e+09,7.709016e+08,BOLT,2017-10-30 16:03:52,https://www.google.com/,25231360,2017-10-31 01:00:30,Mozilla/5.0 (Windows NT 6.0; WOW64; rv:52.0) G...,7ac34a3b-f70a-415a-a1dd-9b01a5e75bd6,Guest,www.google.com
2110,2.605384e+09,1.281296e+09,BOLT,2017-10-30 19:18:19,None,24903680,2017-10-31 01:00:42,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,6655d552-18a0-4a5a-a0aa-7c7db41b679d,Guest,None
2117,2.984837e+09,1.565263e+09,BOLT,2017-10-29 16:31:40,https://www.facebook.com/,24051712,2017-10-31 01:00:39,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,17707223-eab6-43a4-928c-66ac6a64b104,Guest,www.facebook.com


In [24]:
df.loc[55, 'userAgent']

'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'

In [25]:
df.userAgent.head(25)

0     Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1     Mozilla/5.0 (Linux; Android 6.0.1; SM-S327VL B...
2     Mozilla/5.0 (Linux; Android 7.0; SAMSUNG-SM-G9...
3     Mozilla/5.0 (Linux; Android 7.0; SM-G955U Buil...
4     Mozilla/5.0 (Linux; Android 6.0.1; SM-J327P Bu...
5     Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_3 like...
6     Mozilla/5.0 (Linux; Android 7.1.1; SM-N950U Bu...
7     Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG-SM-...
8     Mozilla/5.0 (Linux; Android 7.0; SM-G955U Buil...
9     Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like M...
10    Mozilla/5.0 (Linux; Android 7.0; SAMSUNG SM-G9...
11    Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_3 like...
12    Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_3 like...
13    Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_3 like...
14    Mozilla/5.0 (iPhone; CPU iPhone OS 11_0_3 like...
15    Mozilla/5.0 (Linux; Android 5.1.1; KFFOWI Buil...
16    Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X)...
17    Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG

In [26]:
df.loc[df.userAgent.str.contains('Mozilla')].shape

(12915, 11)

In [27]:
df.columns

Index(['authorKey', 'contentKey', 'pathRoot', 'publishedDate', 'referrer',
       'siteKey', 'timeStamp', 'userAgent', 'userId', 'userStatus',
       'referrer_domain'],
      dtype='object')